In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # Warnin 방지용
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from matplotlib import font_manager, rc

%matplotlib inline
#sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(12,9)

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

In [3]:
factor = pd.read_csv('data2/dw_fire_fighting.csv', encoding='ansi')
factor.head()

,fire_date,city,district,district_code,fire_type,type_code,fire_source,fire_source_code,fire_factor,fire_factor_code,casualties,dead,injury,property_damage,place,place_code
0,2019-01,부산광역시,부산진구,D07,"건축,구조물",T05,작동기기,S02,전기적 요인,F05,0,0,0,45,기타,P05
1,2019-01,부산광역시,남구,D04,"건축,구조물",T05,미상,S04,미상,F04,0,0,0,2997,공동주택,P02
2,2019-01,부산광역시,사상구,D09,"건축,구조물",T05,미상,S04,미상,F04,0,0,0,0,단독주택,P07
3,2019-01,부산광역시,강서구,D01,"건축,구조물",T05,"담뱃불, 라이터불",S07,부주의,F01,0,0,0,113,기타,P05
4,2019-01,부산광역시,강서구,D01,"건축,구조물",T05,작동기기,S02,전기적 요인,F05,0,0,0,65,단독주택,P07


In [4]:
district = [] # 구
district_code = [] # 구 코드
for c in factor['district']:
    if c not in district:
        district.append(c)
for d in factor['district_code']:
    if d not in district_code:
        district_code.append(d)

district_sum = [] # 구 발생건수
for i in range(len(district)):
    d = factor[factor.district == district[i]]   
    ds = d['district'].count()
    district_sum.append(ds)
    
# 구 피해규모
d_damage_sum = []
for i in range(len(district)):
    d = factor[factor.district == district[i]]
    ds = d['property_damage'].sum()
    d_damage_sum.append(ds)
    
district_df = pd.DataFrame({
    'district': district,
    'district_code':district_code,
    'occur_cnt' : district_sum
})

d_damage_df = pd.DataFrame({
    'district': district,
    'district_code':district_code,
    'district_property_damage_k': d_damage_sum
})


In [5]:
import folium
import json

In [6]:
geo_path = "./busan_gu.json"
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [7]:
# 지역별 화재발생건수 맵
district_df.to_csv('map/all_district_occur_cnt.csv')
df_cnt = pd.read_csv('map/all_district_occur_cnt.csv', index_col=1)

map = folium.Map(location = [35.1856505, 129.1], zoom_start=11)
map.choropleth(geo_data = geo_str,
               data = df_cnt['occur_cnt'],
               columns = [df_cnt.index, df_cnt['occur_cnt']],
                          fill_color = 'PuRd', 
                          key_on='feature.id',
              legend_name='화재발생건수')
    #map.save('map/%s_district_map.html'%(place))
map

In [8]:
# 지역별 재산피해규모 맵
d_damage_df.to_csv('map/all_district_damage.csv')
df_damage = pd.read_csv('map/all_district_damage.csv', index_col=1)

map = folium.Map(location = [35.1856505, 129.1], zoom_start=11)
choropleth = map.choropleth(geo_data = geo_str,
               data = df_damage['district_property_damage_k'],
               columns = [df_damage.index, df_damage['district_property_damage_k']],
                          fill_color = 'PuRd', 
                          key_on='feature.id',
                           legend_name="재산피해규모")

    #map.save('map/%s_district_map.html'%(place))
map

In [9]:
Factory_geoData = pd.read_csv('data2/dw_factory.csv', encoding='utf8', engine='python')

In [10]:
map_FT = folium.Map(location = [35.1856505, 129.1], zoom_start = 15)

In [11]:
for i, store in Factory_geoData.iterrows():
    folium.Marker(location = [store['latitude'], store['longitude']],
                 popup=store['district'], icon=folium.Icon(color = 'red',
                                                          icon = 'star')).add_to(map_FT)

In [12]:
map_FT.save('map/map_FT.html')

KeyboardInterrupt: 

In [ ]:
import webbrowser
webbrowser.open('map/map_FT.html')